## Part0 - Personal Data

In [1]:
Name = 'Radin Cheraghi'
Student_Number = '401105815'

## Part1 - Decision Tree Implementation (60 Points)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

In this part, we are going to implement the DecisionTree class from scratch. You are not allowed to use sklearn or any sklearn-like packages which have a built-in implementation of DecisionTree. You are expected to learn about the inner working of decision trees.

### 1.1 Dataset

We will use the *Pima Indians Diabetes* dataset to evaluate our implementation. This dataset consists of 768 records of patients and the goal is to predict whether or not a patient has diabetes.


If you use google colab you need to upload your kaggle.json file. If you want to continue locally you need to download the dataset from [here](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database) and unzip it in *dataset* directory.

In [3]:
!cp kaggle.json /root/.kaggle/
!kaggle datasets download -d uciml/pima-indians-diabetes-database
!unzip pima-indians-diabetes-database.zip

cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 12.2MB/s]
Archive:  pima-indians-diabetes-database.zip
  inflating: diabetes.csv            


In [5]:
df = pd.read_csv('./diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### 1.2 Model Implementation (45 Points)

As you know, in each node of the decision tree we need to choose among all features the best one. One can use different criteria to rank features but here we will use Information Gain. Complete the following functions to use later in the process of learning the decision tree.

In [6]:
# (10 Points)
def entropy(y: pd.Series):
    """
    return the entropy of input
    """
    probabilities = y.value_counts(normalize=True)
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9))
    return entropy

def information_gain(x: pd.Series, y: pd.Series):
    """
    return the information gain of x
    """
    H_y = entropy(y)
    weighted_entropy = 0.0
    for value in x.unique():
        subset = y[x == value]
        weighted_entropy += (len(subset) / len(y)) * entropy(subset)
    return H_y - weighted_entropy

def information_gains(X: pd.DataFrame, y: pd.Series):
    """
    return the information gain of all features
    """
    gains = {}
    for feature in X.columns:
        gains[feature] = information_gain(X[feature], y)
    return gains

To implement decision tree structure we the use following class. Each node in the tree is an instance of class `Node` which is capable of predicting and fitting.

- In the `fit` function this node gets features and labels from its father and using information gain decides which feature to use. Also based on the decided class it will create its children and call their fit function passing relevant features and labels.
- In the `predict` function this node gets features as input and based on its best_feature decides on this input. If this node is a leaf, it will return the decision imediatly and if it's not a leaf, it will return the prediction of its decided child.

In [7]:
# (35 Points)
MAX_DEPTH = 10
class Node:
    def __init__(self, depth):
        self.depth = depth
        self.children = []
        self.best_feature = None
        self.feature_value = None
        self.result = None

    def _is_leaf(self):
        return len(self.children) == 0

    def fit(self, X_train, y_train):
        """
        learn the best_feature and create the children of this node
        """
        self.result = y_train.mode()[0]

        if self.depth >= MAX_DEPTH or len(y_train.unique()) == 1 or X_train.shape[1] == 0:
            return
        gains = information_gains(X_train, y_train)
        self.best_feature = max(gains, key=gains.get)

        best_gain = gains[self.best_feature]

        if best_gain <= 0:
            return

        feature_vals = X_train[self.best_feature].unique()

        for value in feature_vals:
            subset_X = X_train[X_train[self.best_feature] == value].drop(columns=self.best_feature)
            subset_y = y_train[X_train[self.best_feature] == value]
            child = Node(depth=self.depth + 1)
            child.feature_value = value
            child.fit(subset_X, subset_y)
            self.children.append(child)

        if len(self.children) == 0:
            return


    def predict(self, X: pd.DataFrame):
        """
        Predict the class labels for X.
        If this is a leaf node, simply return the stored result.
        If not, delegate the prediction to the appropriate child node based on the best_feature's value.
        """
        if isinstance(X, pd.Series):
            X = pd.DataFrame([X])

        predictions = []
        for _, row in X.iterrows():
            predictions.append(self._predict_single(row))
        return pd.Series(predictions, index=X.index)

    def _predict_single(self, row: pd.Series):
        if self._is_leaf():
            return self.result

        row_value = row[self.best_feature]
        for child in self.children:
            if child.feature_value == row_value:
                return child._predict_single(row)

        return self.result

    def __str__(self) -> str:
        x = ""
        for child in self.children:
            x += str(child) + '\n'
        return f'{self.depth}, {self.best_feature}, {self.feature_value}, {self.result}, ' + x + '\n'

### 1.3 Training & Testing (15 Points)

Now we can learn a decision tree to classify our dataset.

In [15]:
TEST_SIZE = 0.3
RANDOM_STATE = 0

X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# Create the root of the tree
root = Node(depth=0)
root.fit(X_train, y_train)

# Make predictions
y_pred = root.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print("Decision Tree (from scratch) Accuracy: ", acc)

# Optional: Print the (sub)tree for inspection
# print(root)

Decision Tree (from scratch) Accuracy:  0.6406926406926406
